# 📘 Pre-entrega

**Nombre del alumno:** Sanvido Gonzalo sebastian


## 🧩 Etapa 1: Recopilación y Preparación de Datos
**Objetivo:** Demostrar habilidades en Python, familiaridad con el entorno de trabajo y conocimientos básicos sobre manipulación de datos.

### 1️⃣ Carga de datos

In [99]:
import pandas as pd
import numpy as np

In [100]:
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
# Verificar que los archivos csv se encuentren en la carpeta datasets
import os
os.listdir("/content/drive/MyDrive/datasets")

['ventas.csv',
 'marketing.csv',
 'clientes.csv',
 'marketing_clean.csv',
 'clientes_clean.csv',
 'ventas_clean.csv']

In [102]:
# Definimos las rutas de los datasets.

ruta_ventas = "/content/drive/MyDrive/datasets/ventas.csv"
ruta_clientes = "/content/drive/MyDrive/datasets/clientes.csv"
ruta_marketing = "/content/drive/MyDrive/datasets/marketing.csv"

# Cargamos los CSV como DataFrames.
ventas = pd.read_csv(ruta_ventas)
clientes = pd.read_csv(ruta_clientes)
marketing = pd.read_csv(ruta_marketing)

# Validamos formas para comprobar que se cargaron correctamente.
print("ventas.shape ->", ventas.shape)
print("clientes.shape ->", clientes.shape)
print("marketing.shape ->", marketing.shape)

# Mostramos las primeras filas de cada dataset para corroborar estructura de columnas.
display(ventas.head(3))
display(clientes.head(3))
display(marketing.head(3))

ventas.shape -> (3035, 6)
clientes.shape -> (567, 5)
marketing.shape -> (90, 6)


,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.0,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.0,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.0,02/01/2024,Electrodomésticos


,id_cliente,nombre,edad,ciudad,ingresos
0,1,Aloysia Screase,44,Mar del Plata,42294.68
1,2,Kristina Scaplehorn,25,Posadas,24735.04
2,3,Filip Castagne,50,Resistencia,35744.85


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.81,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.40,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.54,28/03/2024,20/04/2024


### 2️⃣ Análisis exploratorio inicial

In [103]:
def eda(df, nombre):
    print(f"=== {nombre} ===")
    print(f"Dimensiones del DataFrame: Filas={df.shape[0]}, Columnas={df.shape[1]}\n")

    print("info:")
    df.info()
    print("\n")

    print("columnas:", list(df.columns))
    print("dtypes:")
    print(df.dtypes)

    print("\nNulos por columna:")
    print(df.isna().sum())

    print("\nNulos por columna en %:")
    print( (((df.isna().sum() / df.shape[0]) * 100 ).round(2)).astype('str') +" %"  )

    print("\nDuplicados totales:")
    print(df.duplicated( keep=False ).sum())

    print("\nPrimeras filas:")
    display(df.head(3))

    print("\nDescribe (numérico):")
    display(df.describe(include='number'))
    print("-"*100)

    print("\nDescribe (categórico):")
    display(df.describe(include='object'))
    print("-"*100)

In [104]:
eda(ventas, "ventas")

=== ventas ===
Dimensiones del DataFrame: Filas=3035, Columnas=6

info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3035 entries, 0 to 3034
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_venta     3035 non-null   int64  
 1   producto     3035 non-null   object 
 2   precio       3033 non-null   object 
 3   cantidad     3033 non-null   float64
 4   fecha_venta  3035 non-null   object 
 5   categoria    3035 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 142.4+ KB


columnas: ['id_venta', 'producto', 'precio', 'cantidad', 'fecha_venta', 'categoria']
dtypes:
id_venta         int64
producto        object
precio          object
cantidad       float64
fecha_venta     object
categoria       object
dtype: object

Nulos por columna:
id_venta       0
producto       0
precio         2
cantidad       2
fecha_venta    0
categoria      0
dtype: int64

Nulos por columna en %:
id_venta       

,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro decorativo,$69.94,5.0,02/01/2024,Decoración
1,811,Lámpara de mesa,$105.10,5.0,02/01/2024,Decoración
2,1156,Secadora,$97.96,3.0,02/01/2024,Electrodomésticos



Describe (numérico):


,id_venta,cantidad
count,3035.000000,3033.000000
mean,1499.851400,6.496538
std,866.465379,3.457250
min,1.000000,1.000000
25%,748.500000,3.000000
50%,1502.000000,7.000000
75%,2249.500000,9.000000
max,3000.000000,12.000000


----------------------------------------------------------------------------------------------------

Describe (categórico):


,producto,precio,fecha_venta,categoria
count,3035,3033,3035,3035
unique,30,2590,364,3
top,Lámpara de mesa,$76.32,06/04/2024,Decoración
freq,181,5,24,1015


----------------------------------------------------------------------------------------------------


In [105]:
eda(clientes, "clientes")

=== clientes ===
Dimensiones del DataFrame: Filas=567, Columnas=5

info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id_cliente  567 non-null    int64  
 1   nombre      567 non-null    object 
 2   edad        567 non-null    int64  
 3   ciudad      567 non-null    object 
 4   ingresos    567 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 22.3+ KB


columnas: ['id_cliente', 'nombre', 'edad', 'ciudad', 'ingresos']
dtypes:
id_cliente      int64
nombre         object
edad            int64
ciudad         object
ingresos      float64
dtype: object

Nulos por columna:
id_cliente    0
nombre        0
edad          0
ciudad        0
ingresos      0
dtype: int64

Nulos por columna en %:
id_cliente    0.0 %
nombre        0.0 %
edad          0.0 %
ciudad        0.0 %
ingresos      0.0 %
dtype: object

Duplicados totales:
0

,id_cliente,nombre,edad,ciudad,ingresos
0,1,Aloysia Screase,44,Mar del Plata,42294.68
1,2,Kristina Scaplehorn,25,Posadas,24735.04
2,3,Filip Castagne,50,Resistencia,35744.85



Describe (numérico):


,id_cliente,edad,ingresos
count,567.000000,567.000000,567.000000
mean,284.000000,37.940035,34668.739012
std,163.823075,10.202885,12974.531446
min,1.000000,20.000000,170.290000
25%,142.500000,30.000000,26015.240000
50%,284.000000,37.000000,35066.830000
75%,425.500000,43.000000,42457.100000
max,567.000000,81.000000,88053.010000


----------------------------------------------------------------------------------------------------

Describe (categórico):


,nombre,ciudad
count,567,567
unique,567,12
top,Micah Matis,Mar del Plata
freq,1,63


----------------------------------------------------------------------------------------------------


In [106]:
eda(marketing, "marketing")

=== marketing ===
Dimensiones del DataFrame: Filas=90, Columnas=6

info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_campanha   90 non-null     int64  
 1   producto      90 non-null     object 
 2   canal         90 non-null     object 
 3   costo         90 non-null     float64
 4   fecha_inicio  90 non-null     object 
 5   fecha_fin     90 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.3+ KB


columnas: ['id_campanha', 'producto', 'canal', 'costo', 'fecha_inicio', 'fecha_fin']
dtypes:
id_campanha       int64
producto         object
canal            object
costo           float64
fecha_inicio     object
fecha_fin        object
dtype: object

Nulos por columna:
id_campanha     0
producto        0
canal           0
costo           0
fecha_inicio    0
fecha_fin       0
dtype: int64

Nulos por columna en %:


,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.81,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.40,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.54,28/03/2024,20/04/2024



Describe (numérico):


,id_campanha,costo
count,90.000000,90.000000
mean,45.500000,4.928667
std,26.124701,0.947750
min,1.000000,2.950000
25%,23.250000,4.372500
50%,45.500000,4.900000
75%,67.750000,5.562500
max,90.000000,7.390000


----------------------------------------------------------------------------------------------------

Describe (categórico):


,producto,canal,fecha_inicio,fecha_fin
count,90,90,90,90
unique,30,3,78,78
top,Adorno de pared,TV,25/09/2024,05/05/2024
freq,3,30,2,3


----------------------------------------------------------------------------------------------------


### 3️⃣ Calidad de los datos

In [107]:

def calidad(df, nombre, clave=None):
    """
    Analiza la calidad del DataFrame:
      - Muestra cantidad de nulos por columna.
      - Cuenta filas duplicadas completas.
      - Si se indica una clave, muestra los valores duplicados más frecuentes.
    Parámetros:
      df: DataFrame de pandas que se analizará.
      nombre: texto descriptivo del DataFrame (ejemplo: 'VENTAS').
      clave: (opcional) nombre de la columna para buscar duplicados específicos.
    """

    # -------------------------------------------------
    # Mostrar título descriptivo con el nombre del DF
    # -------------------------------------------------
    print(f"### {nombre}")

    # -------------------------------------------------
    # Mostrar cantidad de valores nulos por columna
    # -------------------------------------------------
    # df.isna() devuelve un DataFrame booleano con True donde hay NaN.
    # .sum() cuenta los True (o sea, los nulos) por columna.
    # .to_frame("nulos") convierte el resultado en un DataFrame con una columna llamada 'nulos'.
    display(df.isna().sum().to_frame("nulos"))

    # -------------------------------------------------
    # Contar filas duplicadas completas
    # -------------------------------------------------
    # df.duplicated(keep=False) marca como True todas las filas que tienen otra igual.
    # keep=False significa que marca todas las copias, no solo una.
    # .sum() cuenta cuántas filas están repetidas.
    dup_rows = df.duplicated(keep=False).sum()
    print("Filas duplicadas (exactas):", dup_rows)

    # -------------------------------------------------
    # Si se especificó una columna clave válida, analizar duplicados por esa columna
    # -------------------------------------------------
    # if clave analiza que clave no sea None
    # and (y)
    if clave and clave in df.columns:
    # clave in df.columns-- >que clave sea una columna existente dentro de las columnas del dataframe
    # si no le paso ninguna columna no va a querer encontrar duplicados por columna
    # y si me equivoco y le paso una columna que no existe en el dataframe, tampoco ingresara al if.
        # Contar cuántas filas tienen valores repetidos en esa columna
        dup_key = df[clave].duplicated(keep=False).sum()
        print(f"Duplicados por clave '{clave}':", dup_key)

        # Si existen duplicados, mostrar cuáles son los valores más repetidos
        if dup_key > 0:
            # Filtrar filas donde esa clave esté duplicada
            # df[clave].duplicated(keep=False) devuelve True donde el valor se repite
            duplicados_ordenados = (
                df[df[clave].duplicated(keep=False)][clave]
                .value_counts()                # Cuenta cuántas veces aparece cada valor
                .sort_values(ascending=False)   # Ordena de mayor a menor (más duplicados arriba)
            )

            print("\n🔁 Top valores duplicados más frecuentes:")
            # Mostrar solo los primeros 10 (los más repetidos)
            display(duplicados_ordenados.head(10))
        else:
            print(f"No se encontraron duplicados en la clave '{clave}'.")
    else:
        # Si la clave no fue pasada o no existe en el DataFrame
        if clave:
            print(f"La clave '{clave}' no existe en el DataFrame.")
        else:
            print("No se indicó una clave para analizar duplicados por columna.")
#fin de def calidad


In [108]:
calidad(ventas, "ventas", "id_venta")

### ventas


,nulos
id_venta,0
producto,0
precio,2
cantidad,2
fecha_venta,0
categoria,0


Filas duplicadas (exactas): 70
Duplicados por clave 'id_venta': 70

🔁 Top valores duplicados más frecuentes:


,count
id_venta,
56,2
421,2
424,2
1868,2
2545,2
2778,2
145,2
300,2
439,2


In [109]:
calidad(clientes, "clientes", "id_cliente")

### clientes


,nulos
id_cliente,0
nombre,0
edad,0
ciudad,0
ingresos,0


Filas duplicadas (exactas): 0
Duplicados por clave 'id_cliente': 0
No se encontraron duplicados en la clave 'id_cliente'.


In [110]:
calidad(marketing,"marketing","id_campanha")

### marketing


,nulos
id_campanha,0
producto,0
canal,0
costo,0
fecha_inicio,0
fecha_fin,0


Filas duplicadas (exactas): 0
Duplicados por clave 'id_campanha': 0
No se encontraron duplicados en la clave 'id_campanha'.


## 🧹 Etapa 2: Preprocesamiento y Limpieza de Datos
**Objetivo:** Demostrar conocimiento de las técnicas de limpieza y transformación de datos.

### 4️⃣ Limpieza de datos

In [111]:
ventas_clean = ventas.copy( deep =True )
clientes_clean = clientes.copy(deep =True)
marketing_clean = marketing.copy(deep =True)

In [112]:
ventas_clean = ventas_clean.drop_duplicates()

In [113]:
eliminados = ventas.shape[0] - ventas_clean.shape[0]

print(f"Se eliminaron {eliminados} filas duplicadas en ventas_clean.")

Se eliminaron 35 filas duplicadas en ventas_clean.


In [114]:
def normalizar_texto(df):
    for col in df.select_dtypes(include="object").columns:
        # Se agrupan las operaciones entre paréntesis () para escribirlas en varias líneas
        # Python evalúa todo el bloque como una única expresión.
        df[col] = (
            df[col]
            .astype(str)                              # Convierte cualquier tipo a string
            # .astype(str)  → convierte todo a texto; no tiene parámetros adicionales.
            .str.strip()                               # Elimina espacios al inicio y final
            # .str.strip() no necesita argumentos; borra espacios en blanco por defecto.
            .str.replace(r"[\u200b\t\r\n]", "", regex=True)
            # .str.replace(patron, reemplazo, regex=True)
            #   patron: expresión regular que busca caracteres invisibles (\u200b, tabulaciones, saltos)
            #   reemplazo: ""  → los elimina
            #   regex=True indica que 'patron' es una expresión regular.
            .str.replace(" +", " ", regex=True)
            # reemplaza "uno o más espacios consecutivos" por un solo espacio
            .str.title()                               # Convierte a Título: "juan pérez" → "Juan Pérez"
        )

        #df[col] = df[col].astype(str).str.strip().str.replace(r"[\u200b\t\r\n]", "", regex=True).str.replace(" +", " ", regex=True).str.title()
    return df

In [115]:
for df in [ventas_clean, clientes_clean, marketing_clean]:
    for col in df.columns:
        if "fecha" in col.lower():  # detecta columnas con la palabra "fecha"
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

In [116]:
for df,nombre in zip([ventas_clean, clientes_clean, marketing_clean],["ventas","cliente","marketing"]):
    print(f'df {nombre.upper()} \n')
    print(df.dtypes)
    print("-"*100)

df VENTAS 

id_venta                int64
producto               object
precio                 object
cantidad              float64
fecha_venta    datetime64[ns]
categoria              object
dtype: object
----------------------------------------------------------------------------------------------------
df CLIENTE 

id_cliente      int64
nombre         object
edad            int64
ciudad         object
ingresos      float64
dtype: object
----------------------------------------------------------------------------------------------------
df MARKETING 

id_campanha              int64
producto                object
canal                   object
costo                  float64
fecha_inicio    datetime64[ns]
fecha_fin       datetime64[ns]
dtype: object
----------------------------------------------------------------------------------------------------


In [117]:
for df in [ventas_clean, clientes_clean, marketing_clean]:
    df = normalizar_texto(df)

In [118]:
ventas_clean.head(3)

,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro Decorativo,$69.94,5.0,2024-01-02,Decoración
1,811,Lámpara De Mesa,$105.10,5.0,2024-01-02,Decoración
2,1156,Secadora,$97.96,3.0,2024-01-02,Electrodomésticos


In [119]:
if "precio" in ventas_clean.columns:
    # convierto a texto, aplico funciones de texto replace sin expresion regular, quito espacios
    ventas_clean["precio"] = ventas_clean["precio"]\
      .astype(str)\
      .str.replace("$", "", regex=False)\
      .str.replace(",", "", regex=False)\
      .str.strip()



In [120]:
ventas_clean["precio"] = pd.to_numeric(ventas_clean["precio"], errors="coerce")

In [121]:
# convierto a entero las cantidades
if "cantidad" in ventas_clean.columns:
    ventas_clean["cantidad"] = pd.to_numeric( ventas_clean["cantidad"], errors="coerce")\
    .astype("Int64")

In [125]:
ventas_clean.isna().sum()

,0
id_venta,0
producto,0
precio,2
cantidad,2
fecha_venta,0
categoria,0


In [127]:
# busco los nulos
mask = (ventas_clean["precio"].isna()) | (ventas_clean["cantidad"].isna())
ventas_clean[mask]

,id_venta,producto,precio,cantidad,fecha_venta,categoria
136,627,Elementos De Cerámica,NaN,<NA>,2024-01-17,Decoración
139,2171,Parlantes Bluetooth,NaN,<NA>,2024-01-17,Electrónica


In [128]:
# elimino los nulos, son dos regitros
ventas_clean = ventas_clean.dropna(axis= 0 )

In [129]:
for df,nombre in zip([ventas_clean, clientes_clean, marketing_clean],["ventas","cliente","marketing"]):
    print(f'df {nombre.upper()} \n')
    print(df.dtypes)
    print("-"*100)

df VENTAS 

id_venta                int64
producto               object
precio                float64
cantidad                Int64
fecha_venta    datetime64[ns]
categoria              object
dtype: object
----------------------------------------------------------------------------------------------------
df CLIENTE 

id_cliente      int64
nombre         object
edad            int64
ciudad         object
ingresos      float64
dtype: object
----------------------------------------------------------------------------------------------------
df MARKETING 

id_campanha              int64
producto                object
canal                   object
costo                  float64
fecha_inicio    datetime64[ns]
fecha_fin       datetime64[ns]
dtype: object
----------------------------------------------------------------------------------------------------


In [130]:
ventas_clean.to_csv("/content/drive/MyDrive/datasets/ventas_clean.csv", index=False)
clientes_clean.to_csv("/content/drive/MyDrive/datasets/clientes_clean.csv", index=False)
marketing_clean.to_csv("/content/drive/MyDrive/datasets/marketing_clean.csv", index=False)

print("✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv")

✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv


In [131]:
def reporte_calidad_global(dfs, nombres):
    """
    Crea un resumen de calidad de varios DataFrames.

    Parámetros:
      dfs: lista de DataFrames (por ejemplo [ventas_clean, clientes_clean, marketing_clean])
      nombres: lista de nombres correspondientes (["VENTAS", "CLIENTES", "MARKETING"])
    """
    resumen = []
    #zip-->es una función incorporada de Python que une elementos de dos (o más) iterables
    # —por ejemplo, listas, tuplas o cualquier objeto iterable— en pares ordenados.
    for df, nombre in zip(dfs, nombres):
        nulos = df.isna().sum().sum()                    # Total de valores nulos, no por columnas sino total, por eso el doble sum
        duplicados = df.duplicated(keep=False).sum()     # Total de filas duplicadas
        columnas = len(df.columns)                       # Cantidad de columnas
        filas = len(df)                                  # Cantidad de registros

        resumen.append({
            "Dataset": nombre,
            "Filas": filas,
            "Columnas": columnas,
            "Nulos totales": nulos,
            "Duplicados": duplicados,
        })

    reporte = pd.DataFrame(resumen)
    #display(reporte)
    return reporte

In [132]:
print(reporte_calidad_global([ventas, clientes, marketing], ["VENTAS Original", "CLIENTES Original", "MARKETING Original"]))
print(reporte_calidad_global([ventas_clean, clientes_clean, marketing_clean],["VENTAS Copia   ", "CLIENTES Copia   ", "MARKETING Copia   "]))

              Dataset  Filas  Columnas  Nulos totales  Duplicados
0     VENTAS Original   3035         6              4          70
1   CLIENTES Original    567         5              0           0
2  MARKETING Original     90         6              0           0
              Dataset  Filas  Columnas  Nulos totales  Duplicados
0     VENTAS Copia      2998         6              0           0
1   CLIENTES Copia       567         5              0           0
2  MARKETING Copia        90         6              0           0


In [133]:
def comparacion_columnas(df1, df2, nombres):
    """
    Compara las columnas de dos DataFrames y muestra las que son diferentes."""
    df =pd.concat([df1.dtypes, df2.dtypes], axis=1)
    df.columns = [nombres[0], nombres[1]]
    df['diferencia'] = np.where(df[nombres[0]] != df[nombres[1]], "Diferente", "Igual")
    return df



In [134]:
comparacion_columnas(ventas, ventas_clean, ["ventas", "ventas_clean"])

,ventas,ventas_clean,diferencia
id_venta,int64,int64,Igual
producto,object,object,Igual
precio,object,float64,Diferente
cantidad,float64,Int64,Diferente
fecha_venta,object,datetime64[ns],Diferente
categoria,object,object,Igual


In [135]:
comparacion_columnas(clientes, clientes_clean, ["clientes", "clientes_clean"])

,clientes,clientes_clean,diferencia
id_cliente,int64,int64,Igual
nombre,object,object,Igual
edad,int64,int64,Igual
ciudad,object,object,Igual
ingresos,float64,float64,Igual


In [136]:
comparacion_columnas(marketing, marketing_clean, ["marketing", "marketing_clean"])

,marketing,marketing_clean,diferencia
id_campanha,int64,int64,Igual
producto,object,object,Igual
canal,object,object,Igual
costo,float64,float64,Igual
fecha_inicio,object,datetime64[ns],Diferente
fecha_fin,object,datetime64[ns],Diferente


### 5️⃣ Transformación de datos

In [137]:
# TODO: Aplicar filtros y transformaciones para crear una tabla de ventas
# que muestre solo los productos con alto rendimiento. calcular el percentil 80
# y filtrar los productos que superen ese umbral en ventas.
#quantile(0.8)
# Sugerencia: usar .query() o condiciones con operadores lógicos.

2. Transformación de Datos: Aplicar filtros y transformaciones para crear una
tabla de ventas que muestre solo los productos con alto rendimiento.
3. Agregación: Resumir las ventas por categoría de producto y analizar los
ingresos generados.
4. Integración de Datos: Combinar los sets de datos de ventas y marketing
para obtener una visión más amplia de las tendencias.

In [138]:
ventas_clean.head(3)

,id_venta,producto,precio,cantidad,fecha_venta,categoria
0,792,Cuadro Decorativo,69.94,5,2024-01-02,Decoración
1,811,Lámpara De Mesa,105.10,5,2024-01-02,Decoración
2,1156,Secadora,97.96,3,2024-01-02,Electrodomésticos


In [139]:
ventas_clean['total'] = ventas_clean['precio'] * ventas_clean['cantidad']
ventas_clean.head(3)

,id_venta,producto,precio,cantidad,fecha_venta,categoria,total
0,792,Cuadro Decorativo,69.94,5,2024-01-02,Decoración,349.7
1,811,Lámpara De Mesa,105.10,5,2024-01-02,Decoración,525.5
2,1156,Secadora,97.96,3,2024-01-02,Electrodomésticos,293.88


### 6️⃣ Agregación

In [140]:
# agrupacion por producto y saco agregaciones
resumen_prod = ventas_clean.groupby(by='producto', as_index=False)\
  .agg(ingreso_total=('total', 'sum'), unidades=('cantidad', 'sum'), precio_promedio=('precio', 'mean'), registros=('total', 'size'))\
  .sort_values(by='ingreso_total', ascending=False)

resumen_prod.head(3)

,producto,ingreso_total,unidades,precio_promedio,registros
19,Lámpara De Mesa,82276.38,1112,72.720625,176
3,Auriculares,74175.58,958,76.302727,143
20,Microondas,72562.89,912,79.176000,135


In [144]:
# percentil 80 sobre cantidades
cantidades_percentil_80 = resumen_prod['unidades'].quantile(0.8)
print(f"El percentil 80 de las cantidades es: {cantidades_percentil_80}")

El percentil 80 de las cantidades es: 692.0


In [145]:
# percentil 80 sobre total
ingresos_percentil_80 = resumen_prod['ingreso_total'].quantile(0.8)
print(f"El percentil 80 del total de ventas es: {ingresos_percentil_80:.2f}")

El percentil 80 del total de ventas es: 52518.85


In [146]:
# filtro los productos de mayor ingreso total superior al percentil 80
resumen_prod_alto_rendimiento = resumen_prod[resumen_prod['ingreso_total'] > ingresos_percentil_80].sort_values(by='ingreso_total', ascending=False)
resumen_prod_alto_rendimiento.head()

,producto,ingreso_total,unidades,precio_promedio,registros
19,Lámpara De Mesa,82276.38,1112,72.720625,176
3,Auriculares,74175.58,958,76.302727,143
20,Microondas,72562.89,912,79.176000,135
5,Cafetera,59607.31,765,79.046581,117
9,Cuadro Decorativo,54297.6,726,74.578000,100


In [147]:
# total de ingresos de los productos de alto rendimiento
ingresos_superiores = resumen_prod_alto_rendimiento['ingreso_total'].sum()
# total de ingresos
total_ingresos = resumen_prod['ingreso_total'].sum()
# calculo de porcentaje, cuanto representan
porcentaje_ingresos = (ingresos_superiores / total_ingresos) * 100
print(f"El porcentaje de ingresos superiores al percentil 80 es: {porcentaje_ingresos:.2f}%")


El porcentaje de ingresos superiores al percentil 80 es: 27.06%


In [148]:
cantidad_productos = resumen_prod.shape[0]
print(f"La cantidad total de productos es: {cantidad_productos}")

La cantidad total de productos es: 30


In [149]:
cantidad_productos_alto_rendimiento = resumen_prod_alto_rendimiento.shape[0]
print(f"La cantidad de productos con ingresos superiores al percentil 80 es: {cantidad_productos_alto_rendimiento}")
print(f"El porcentaje de productos con ingresos superiores al percentil 80 es: {(cantidad_productos_alto_rendimiento/cantidad_productos)*100:.2f}%")

La cantidad de productos con ingresos superiores al percentil 80 es: 6
El porcentaje de productos con ingresos superiores al percentil 80 es: 20.00%


In [150]:
# agregar columnas, total acumulado sobre los productos ordenados en orden decreciente por total de ventas
resumen_prod['total_acumulado']= resumen_prod['ingreso_total'].cumsum()
# agregar columna total general de ventas
resumen_prod['total_general'] = resumen_prod['ingreso_total'].sum()
# porcentaje acumulado en porcentaje
resumen_prod['porcentaje_acumulado'] = ((resumen_prod['total_acumulado'] / resumen_prod['total_general']) * 100).round(2)
resumen_prod.head(30)

,producto,ingreso_total,unidades,precio_promedio,registros,total_acumulado,total_general,porcentaje_acumulado
19,Lámpara De Mesa,82276.38,1112,72.720625,176,82276.38,1467093.52,5.61
3,Auriculares,74175.58,958,76.302727,143,156451.96,1467093.52,10.66
20,Microondas,72562.89,912,79.176000,135,229014.85,1467093.52,15.61
5,Cafetera,59607.31,765,79.046581,117,288622.16,1467093.52,19.67
9,Cuadro Decorativo,54297.6,726,74.578000,100,342919.76,1467093.52,23.37
26,Smartphone,54132.44,665,81.398416,101,397052.2,1467093.52,27.06
25,Secadora,52115.45,696,73.338900,100,449167.65,1467093.52,30.62
16,Jarrón Decorativo,51130.88,672,74.534000,100,500298.53,1467093.52,34.1
4,Batidora,50979.2,672,77.537200,100,551277.73,1467093.52,37.58
24,Rincón De Plantas,50456.45,691,74.823861,101,601734.18,1467093.52,41.02


In [151]:
# mascara booleana, los mejores productos que representan el 80 % de las ventas
mask = resumen_prod['porcentaje_acumulado'] < 80

In [152]:
cantidad_productos_alto_rendimiento_2 = resumen_prod[mask].shape[0]

In [153]:
print(f"El 80 % de los ingresos es generado por : {(cantidad_productos_alto_rendimiento_2/cantidad_productos)*100:.2f} % de los productos")

El 80 % de los ingresos es generado por : 70.00 % de los productos


In [154]:
ventas_clean.head()

,id_venta,producto,precio,cantidad,fecha_venta,categoria,total
0,792,Cuadro Decorativo,69.94,5,2024-01-02,Decoración,349.7
1,811,Lámpara De Mesa,105.10,5,2024-01-02,Decoración,525.5
2,1156,Secadora,97.96,3,2024-01-02,Electrodomésticos,293.88
3,1372,Heladera,114.35,8,2024-01-02,Electrodomésticos,914.8
4,1546,Secadora,106.21,4,2024-01-02,Electrodomésticos,424.84


In [159]:

# crear columnas de fecha para analizar las ventas por año y mes
ventas_clean['año'] = ventas_clean['fecha_venta'].dt.year
ventas_clean['mes'] = ventas_clean['fecha_venta'].dt.month
ventas_clean['dia'] = ventas_clean['fecha_venta'].dt.day_name()
ventas_clean.head()

ventas_clean.head()

,id_venta,producto,precio,cantidad,fecha_venta,categoria,total,año,mes,dia
0,792,Cuadro Decorativo,69.94,5,2024-01-02,Decoración,349.7,2024,1,Tuesday
1,811,Lámpara De Mesa,105.10,5,2024-01-02,Decoración,525.5,2024,1,Tuesday
2,1156,Secadora,97.96,3,2024-01-02,Electrodomésticos,293.88,2024,1,Tuesday
3,1372,Heladera,114.35,8,2024-01-02,Electrodomésticos,914.8,2024,1,Tuesday
4,1546,Secadora,106.21,4,2024-01-02,Electrodomésticos,424.84,2024,1,Tuesday


In [164]:
# ventas por año y mes, ordenadas descendentes
resumen_prod_2 = ventas_clean.groupby(by=['año','mes']).agg(total_ventas = ('total', 'sum')).sort_values(by='total_ventas', ascending=False)
resumen_prod_2

total_ventas
año  mes              
2024 5       143727.25
     3       136779.15
     1       129604.99
     4       128430.69
     11      119951.79
     8       119680.15
     2       118672.44
     12      117631.94
     7       116229.97
     9       115787.85
     10      112117.13
     6       108480.17

In [167]:
resumen_prod_2['como %'] = (resumen_prod_2['total_ventas'] / resumen_prod_2['total_ventas'].sum() * 100).round(2)
resumen_prod_2

total_ventas  como %
año  mes                      
2024 5       143727.25     9.8
     3       136779.15    9.32
     1       129604.99    8.83
     4       128430.69    8.75
     11      119951.79    8.18
     8       119680.15    8.16
     2       118672.44    8.09
     12      117631.94    8.02
     7       116229.97    7.92
     9       115787.85    7.89
     10      112117.13    7.64
     6       108480.17    7.39

In [168]:
# ventas por dia de la semana
resumen_prod_3 = ventas_clean.groupby(by=['dia']).agg(total_ventas = ('total', 'sum')).sort_values(by='total_ventas', ascending=False)
resumen_prod_3

,total_ventas
dia,
Thursday,220610.75
Wednesday,220477.42
Sunday,219720.77
Saturday,215017.9
Tuesday,200745.02
Monday,199054.71
Friday,191466.95


In [171]:
# mayor venta es los jueves y miercoles
resumen_prod_3['como %'] = (resumen_prod_3['total_ventas'] / resumen_prod_3['total_ventas'].sum() * 100).round(2)
resumen_prod_3

,total_ventas,como %
dia,,
Thursday,220610.75,15.04
Wednesday,220477.42,15.03
Sunday,219720.77,14.98
Saturday,215017.9,14.66
Tuesday,200745.02,13.68
Monday,199054.71,13.57
Friday,191466.95,13.05


In [173]:
# resumen por categoria, los electrodomesticos los que mas ganan
resumen_prod_4 = ventas_clean.groupby(by=['categoria']).agg(total_ventas = ('total', 'sum')).sort_values(by='total_ventas', ascending=False)
resumen_prod_4

,total_ventas
categoria,
Electrodomésticos,505299.63
Electrónica,482577.8
Decoración,479216.09


los archivos utilizados

https://drive.google.com/file/d/12J4AL1f34xbVnXizyiuSWc-hBHz2o2SU/view?usp=drive_link

https://drive.google.com/file/d/1weBzJhPezQ_fgDK432vV9tEV76cwYZLa/view?usp=drive_link

https://drive.google.com/file/d/1NkPmgGKaPWPqpfeZ-WYfHCUKVPIzPtE6/view?usp=drive_link